In [1]:
import tensorflow as tf
import numpy as np
import os, pdb
import cv2
import numpy as np
import random as rn
import threading
import time
from sklearn import metrics
import utils
global n_classes
import triplet_loss as tri


n_classes = 2


def activation(x,name="activation"):
    return tf.nn.swish(x, name=name)
    
def conv2d(name, l_input, w, b, s, p):
    l_input = tf.nn.conv2d(l_input, w, strides=[1,s,s,1], padding=p, name=name)
    l_input = l_input+b

    return l_input

def batchnorm(conv, isTraining, name='bn'):
    return tf.layers.batch_normalization(conv, training=isTraining, name="bn"+name)

def initializer(in_filters, out_filters, name):
    w1 = tf.get_variable(name+"W", [3, 3, in_filters, out_filters], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable(name+"B", [out_filters], initializer=tf.truncated_normal_initializer())
    return w1, b1
  
def residual_block(in_x, in_filters, out_filters, stride, isDownSampled, name, isTraining):
    global ema_gp
    # first convolution layer
    if isDownSampled:
      in_x = tf.nn.avg_pool(in_x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
      
    x = batchnorm(in_x, isTraining, name=name+'FirstBn')
    x = activation(x)
    w1, b1 = initializer(in_filters, in_filters, name+"first_res")
    x = conv2d(name+'r1', x, w1, b1, 1, "SAME")

    # second convolution layer
    x = batchnorm(x, isTraining, name=name+'SecondBn')
    x = activation(x)
    w2, b2 = initializer(in_filters, out_filters, name+"Second_res")
    x = conv2d(name+'r2', x, w2, b2, 1, "SAME")
    
    if in_filters != out_filters:
        difference = out_filters - in_filters
        left_pad = difference // 2
        right_pad = difference - left_pad
        identity = tf.pad(in_x, [[0, 0], [0, 0], [0, 0], [left_pad, right_pad]])
        return x + identity
    else:
        return in_x + x

      
def ResNet(_X, isTraining):
    global n_classes
    w1 = tf.get_variable("initWeight", [7, 7, 3, 64], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable("initBias", [64], initializer=tf.truncated_normal_initializer())
    x = conv2d('conv1', _X, w1, b1, 4, "VALID")
    
    filters_num = [64,96,128]
    block_num = [2,4,3]
    l_cnt = 1
    for i in range(len(filters_num)):
      for j in range(block_num[i]):
          
          if ((j==block_num[i]-1) & (i<len(filters_num)-1)):
            x = residual_block(x, filters_num[i], filters_num[i+1], 2, True, 'ResidualBlock%d'%(l_cnt), isTraining)
            print('[L-%d] Build %dth connection layer %d from %d to %d channels' % (l_cnt, i, j, filters_num[i], filters_num[i+1]))
          else:
            x = residual_block(x, filters_num[i], filters_num[i], 1, False, 'ResidualBlock%d'%(l_cnt), isTraining)
            print('[L-%d] Build %dth residual block %d with %d channels' % (l_cnt,i, j, filters_num[i]))
          l_cnt +=1

    saliency = x
    x_shape = x.get_shape().as_list()
    dense1 = x_shape[1]*x_shape[2]*x_shape[3]
    W = tf.get_variable("featW", [dense1, 128], initializer=tf.truncated_normal_initializer())
    b = tf.get_variable("featB", [128], initializer=tf.truncated_normal_initializer())
    dense1 = tf.reshape(x, [-1, dense1])
    feat = tf.nn.softmax(tf.matmul(dense1, W) + b)
    
    with tf.variable_scope('Final'):
        x = batchnorm(x, isTraining, name='FinalBn')
        x = activation(x)
        wo, bo=initializer(filters_num[-1], n_classes, "FinalOutput")
        x = conv2d('final', x, wo, bo, 1, "SAME")
        saliency = x
        x=tf.reduce_mean(x, [1, 2])

        W = tf.get_variable("FinalW", [n_classes, n_classes], initializer=tf.truncated_normal_initializer())
        b = tf.get_variable("FinalB", [n_classes], initializer=tf.truncated_normal_initializer())

        out = tf.matmul(x, W) + b
                            

    return out, feat, saliency


#==========================================================================
#=============Reading data in multithreading manner========================
#==========================================================================
def read_labeled_image_list(image_list_file, training_img_dir):
    f = open(image_list_file, 'r')
    filenames = []
    labels = []

    for line in f:
        filename, label = line[:-1].split(' ')
        filename = training_img_dir+filename
        filenames.append(filename)
        labels.append(int(label))
        
    return filenames, labels
    
    
def read_images_from_disk(input_queue, size1=64):
    label = input_queue[1]
    fn=input_queue[0]
    file_contents = tf.read_file(input_queue[0])
    example = tf.image.decode_jpeg(file_contents, channels=3)
    
    #example = tf.image.decode_png(file_contents, channels=3, name="dataset_image") # png fo rlfw
    example=tf.image.resize_images(example, [size1,size1])
    return example, label, fn
    
def setup_inputs(sess, filenames, training_img_dir, image_size=64, crop_size=64, isTest=False, batch_size=128):
    
    # Read each image file
    image_list, label_list = read_labeled_image_list(filenames, training_img_dir)

    images = tf.cast(image_list, tf.string)
    labels = tf.cast(label_list, tf.int64)
     # Makes an input queue
    if isTest is False:
        isShuffle = True
        numThr = 4
    else:
        isShuffle = False
        numThr = 1
        
    input_queue = tf.train.slice_input_producer([images, labels], shuffle=isShuffle)
    image, y,fn = read_images_from_disk(input_queue)

    channels = 3
    image.set_shape([None, None, channels])
        
    # Crop and other random augmentations
    if isTest is False:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_saturation(image, .95, 1.05)
        image = tf.image.random_brightness(image, .05)
        image = tf.image.random_contrast(image, .95, 1.05)
        
    image = tf.cast(image, tf.float32)/255.0
    
    image, y,fn = tf.train.batch([image, y, fn], batch_size=batch_size, capacity=batch_size*3, num_threads=numThr, name='labels_and_images')

    tf.train.start_queue_runners(sess=sess)

    return image, y, fn, len(label_list)


batch_size = 64
display_step = 80
learning_rate = tf.placeholder(tf.float32)      # Learning rate to be fed
lr = 1e-3                        # Learning rate start
tst = tf.placeholder(tf.bool)
iter = tf.placeholder(tf.int32)
print('GO!!')


# In[ ]:


# Setup the tensorflow...
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

print("Preparing the training & validation data...")
train_data, train_labels, filelist1, glen1 = setup_inputs(sess, "data/train_wo_progressGAN.txt", "./", batch_size=batch_size)
val_data, val_labels, filelist2, tlen1 = setup_inputs(sess, "data/val_wo_progressGAN.txt", "./", batch_size=1000,isTest=True)
print("Found %d training images, and %d validation images..." % (glen1, tlen1))

max_iter = glen1*15
print("Preparing the training model with learning rate = %.5f..." % (lr))


with tf.variable_scope("ResNet") as scope:
    pred, feat,_ = ResNet(train_data, True)
    scope.reuse_variables()
    valpred, _, saliency = ResNet(val_data, False)


  
with tf.name_scope('Triplet_loss'):
  margin = 0.4
  sialoss = tri.batch_hard_triplet_loss(train_labels, feat, margin, squared=False)
  
  
with tf.name_scope('Loss_and_Accuracy'):
  update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
  with tf.control_dependencies(update_ops):
    t_vars=tf.trainable_variables() 
    #t_vars=[var for var in t_vars if 'Final']
    cost = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=pred)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, var_list=t_vars)
    sia_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(sialoss)
  
  correct_prediction = tf.equal(tf.argmax(pred, 1), train_labels)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  correct_prediction2 = tf.equal(tf.argmax(valpred, 1), val_labels)
  accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
  
  
tf.summary.scalar("Triplet_loss", sialoss)
tf.summary.scalar('Loss', cost)
tf.summary.scalar('Training_Accuracy', accuracy)
tf.summary.scalar('Validation_Accuracy', accuracy2)
# In[ ]:


saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess.run(init)
step = 0
writer = tf.summary.FileWriter("logs/sia/", sess.graph)
summaries = tf.summary.merge_all()

print("We are going to pretrain model susing ResNet based on triplet loss!!!")
start_lr = lr
while (step * batch_size) < max_iter:
    epoch1=np.floor((step*batch_size)/glen1)
    if (((step*batch_size)%glen1 < batch_size) & (lr==1e-4) & (epoch1 >4)):
        lr /= 10
    
    if epoch1 <=1:
        sess.run([sia_optimizer],  feed_dict={learning_rate: lr})
    else:
#         if start_lr == lr:
#             lr = lr /10
        sess.run([optimizer],  feed_dict={learning_rate: lr})
        
    if (step % 15000==1) & (step>15000):
        save_path = saver.save(sess, "checkpoints/tf_deepUD_tri_model_iter" + str(step) + ".ckpt")
        print("Model saved in file at iteration %d: %s" % (step*batch_size,save_path))

    if step>0 and step % display_step == 0:
        # calculate the loss
        loss, acc, summaries_string, sia_val = sess.run([cost, accuracy, summaries, sialoss])
        print("Iter=%d/epoch=%d, Loss=%.6f, Triplet loss=%.6f, Training Accuracy=%.6f, lr=%f" % (step*batch_size, epoch1 ,loss, sia_val, acc, lr))
        writer.add_summary(summaries_string, step)
    
    if step>0 and (step % (display_step*20) == 0):
        rounds = tlen1 // 1000
        #pdb.set_trace()
        valacc=[]
        vis=[]
        tis=[]
        for k in range(rounds):
          a2, vi, ti = sess.run([accuracy2, tf.argmax(valpred, 1), val_labels])
          valacc.append(a2)
          vis.append(vi)
          tis.append(ti)
        tis = np.reshape(np.asarray(tis), [-1])
        vis = np.reshape(np.asarray(vis), [-1])
        precision=metrics.precision_score(tis, vis) 
        recall=metrics.recall_score(tis, vis)
        
        sal, valimg = sess.run([tf.argmin(saliency, axis=3), val_data])
        utils.batchsalwrite(valimg, sal, tis, vis, 'saliency_img/Detected_')

        print("Iter=%d/epoch=%d, Validation Accuracy=%.6f, Precision=%.6f, Recall=%.6f" % (step*batch_size, epoch1 , np.mean(valacc), precision, recall))

  
    step += 1
print("Optimization Finished!")
save_path = saver.save(sess, "checkpoints/tf_deepUD_tri_model.ckpt")
print("Model saved in file: %s" % save_path)






GO!!
Preparing the training & validation data...
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Found 385198 training images, and 10000 validation images...
Preparing the training model with learning rate = 0.00100...
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
[L-1] Build 0th residual block 0 with 64 channels
[L-2] Build 0th connection layer 1 from 64 to 96 channels
[L-3] Build 1th residual block 0 with 96 channels
[L-4] Build 1th residual block 1 with 96 channels
[L-5] Build 1th residual block 2 with 96 channels
[L-6] Build 1th connection layer 3 from 96 to 128 channels
[L-7] Build 2th residual block 0 with 128 channels
[L-8] Build 2th residual block 1 with 128 channels
[L-9] Build 2th residual block 2 with 128 channels
[L-1]

Iter=296960/epoch=0, Loss=3.861104, Triplet loss=1.814214, Training Accuracy=0.421875, lr=0.001000
Iter=302080/epoch=0, Loss=2.987463, Triplet loss=1.814214, Training Accuracy=0.593750, lr=0.001000
Iter=307200/epoch=0, Loss=4.072070, Triplet loss=1.792563, Training Accuracy=0.421875, lr=0.001000
Iter=307200/epoch=0, Validation Accuracy=0.518200, Precision=0.514725, Recall=0.636200
Iter=312320/epoch=0, Loss=3.746872, Triplet loss=1.770019, Training Accuracy=0.500000, lr=0.001000
Iter=317440/epoch=0, Loss=3.593994, Triplet loss=1.814214, Training Accuracy=0.609375, lr=0.001000
Iter=322560/epoch=0, Loss=3.619717, Triplet loss=1.814214, Training Accuracy=0.562500, lr=0.001000
Iter=327680/epoch=0, Loss=3.495907, Triplet loss=1.814214, Training Accuracy=0.515625, lr=0.001000
Iter=332800/epoch=0, Loss=3.309761, Triplet loss=1.770019, Training Accuracy=0.562500, lr=0.001000
Iter=337920/epoch=0, Loss=2.662730, Triplet loss=1.814214, Training Accuracy=0.578125, lr=0.001000
Iter=343040/epoch=0, L

Iter=706560/epoch=1, Loss=4.167546, Triplet loss=1.703728, Training Accuracy=0.437500, lr=0.001000
Iter=711680/epoch=1, Loss=3.670456, Triplet loss=1.725825, Training Accuracy=0.453125, lr=0.001000
Iter=716800/epoch=1, Loss=2.800234, Triplet loss=1.747922, Training Accuracy=0.562500, lr=0.001000
Iter=716800/epoch=1, Validation Accuracy=0.482600, Precision=0.485412, Recall=0.579000
Iter=721920/epoch=1, Loss=2.451875, Triplet loss=1.770014, Training Accuracy=0.609375, lr=0.001000
Iter=727040/epoch=1, Loss=6.455324, Triplet loss=1.814214, Training Accuracy=0.390625, lr=0.001000
Iter=732160/epoch=1, Loss=5.786986, Triplet loss=1.703728, Training Accuracy=0.640625, lr=0.001000
Iter=737280/epoch=1, Loss=4.094543, Triplet loss=1.747922, Training Accuracy=0.593750, lr=0.001000
Iter=742400/epoch=1, Loss=4.592614, Triplet loss=1.725825, Training Accuracy=0.500000, lr=0.001000
Iter=747520/epoch=1, Loss=3.894461, Triplet loss=1.770019, Training Accuracy=0.484375, lr=0.001000
Iter=752640/epoch=1, L

Iter=1111040/epoch=2, Loss=0.112951, Triplet loss=1.792116, Training Accuracy=0.953125, lr=0.001000
Iter=1116160/epoch=2, Loss=0.069822, Triplet loss=1.792116, Training Accuracy=0.968750, lr=0.001000
Iter=1121280/epoch=2, Loss=0.034206, Triplet loss=1.792116, Training Accuracy=1.000000, lr=0.001000
Iter=1126400/epoch=2, Loss=0.019506, Triplet loss=1.792116, Training Accuracy=1.000000, lr=0.001000
Iter=1126400/epoch=2, Validation Accuracy=0.570200, Precision=0.537930, Recall=0.995600
Iter=1131520/epoch=2, Loss=0.058401, Triplet loss=1.792116, Training Accuracy=0.984375, lr=0.001000
Iter=1136640/epoch=2, Loss=0.023764, Triplet loss=1.703728, Training Accuracy=1.000000, lr=0.001000
Iter=1141760/epoch=2, Loss=0.013792, Triplet loss=1.770019, Training Accuracy=1.000000, lr=0.001000
Iter=1146880/epoch=2, Loss=0.050458, Triplet loss=1.792116, Training Accuracy=0.953125, lr=0.001000
Iter=1152000/epoch=2, Loss=0.033117, Triplet loss=1.770019, Training Accuracy=0.984375, lr=0.001000
Iter=1157120

Iter=1515520/epoch=3, Loss=0.025650, Triplet loss=1.814214, Training Accuracy=0.984375, lr=0.001000
Iter=1520640/epoch=3, Loss=0.039426, Triplet loss=1.814214, Training Accuracy=0.968750, lr=0.001000
Iter=1525760/epoch=3, Loss=0.004575, Triplet loss=1.814214, Training Accuracy=1.000000, lr=0.001000
Iter=1530880/epoch=3, Loss=0.006109, Triplet loss=1.770019, Training Accuracy=1.000000, lr=0.001000
Iter=1536000/epoch=3, Loss=0.003200, Triplet loss=1.792116, Training Accuracy=1.000000, lr=0.001000
Iter=1536000/epoch=3, Validation Accuracy=0.854000, Precision=0.786872, Recall=0.971000
Iter=1541120/epoch=4, Loss=0.015334, Triplet loss=1.770019, Training Accuracy=1.000000, lr=0.001000
Iter=1546240/epoch=4, Loss=0.011724, Triplet loss=1.814214, Training Accuracy=1.000000, lr=0.001000
Iter=1551360/epoch=4, Loss=0.025162, Triplet loss=1.770019, Training Accuracy=1.000000, lr=0.001000
Iter=1556480/epoch=4, Loss=0.078872, Triplet loss=1.770019, Training Accuracy=0.968750, lr=0.001000
Iter=1561600

...

Iter=3927040/epoch=10, Loss=0.000001, Triplet loss=0.834673, Training Accuracy=1.000000, lr=0.001000
Iter=3932160/epoch=10, Loss=0.000367, Triplet loss=1.283884, Training Accuracy=1.000000, lr=0.001000
Iter=3937280/epoch=10, Loss=0.000069, Triplet loss=1.328078, Training Accuracy=1.000000, lr=0.001000
Iter=3942400/epoch=10, Loss=0.000023, Triplet loss=0.698818, Training Accuracy=1.000000, lr=0.001000
Iter=3947520/epoch=10, Loss=0.000004, Triplet loss=1.307071, Training Accuracy=1.000000, lr=0.001000
Iter=3952640/epoch=10, Loss=0.000016, Triplet loss=0.532583, Training Accuracy=1.000000, lr=0.001000
Iter=3957760/epoch=10, Loss=0.000002, Triplet loss=0.532583, Training Accuracy=1.000000, lr=0.001000
Iter=3962880/epoch=10, Loss=0.000065, Triplet loss=1.261792, Training Accuracy=1.000000, lr=0.001000
Iter=3968000/epoch=10, Loss=0.000848, Triplet loss=0.598874, Training Accuracy=1.000000, lr=0.001000
Iter=3973120/epoch=10, Loss=0.000004, Triplet loss=0.731457, Training Accuracy=1.000000, lr

Iter=4326400/epoch=11, Loss=0.000000, Triplet loss=1.394369, Training Accuracy=1.000000, lr=0.001000
Iter=4331520/epoch=11, Loss=0.000096, Triplet loss=1.350175, Training Accuracy=1.000000, lr=0.001000
Iter=4336640/epoch=11, Loss=0.000026, Triplet loss=0.775651, Training Accuracy=1.000000, lr=0.001000
Iter=4341760/epoch=11, Loss=0.000009, Triplet loss=1.350175, Training Accuracy=1.000000, lr=0.001000
Iter=4346880/epoch=11, Loss=0.003019, Triplet loss=1.261783, Training Accuracy=1.000000, lr=0.001000
Iter=4352000/epoch=11, Loss=0.000001, Triplet loss=1.372272, Training Accuracy=1.000000, lr=0.001000
Iter=4357120/epoch=11, Loss=0.000031, Triplet loss=0.554680, Training Accuracy=1.000000, lr=0.001000
Iter=4362240/epoch=11, Loss=0.000000, Triplet loss=0.687262, Training Accuracy=1.000000, lr=0.001000
Iter=4367360/epoch=11, Loss=0.000352, Triplet loss=0.620971, Training Accuracy=1.000000, lr=0.001000
Iter=4372480/epoch=11, Loss=0.007198, Triplet loss=1.416466, Training Accuracy=1.000000, lr

Iter=4725760/epoch=12, Loss=0.000003, Triplet loss=1.328078, Training Accuracy=1.000000, lr=0.001000
Iter=4730880/epoch=12, Loss=0.000021, Triplet loss=1.239689, Training Accuracy=1.000000, lr=0.001000
Iter=4736000/epoch=12, Loss=0.001941, Triplet loss=1.350702, Training Accuracy=1.000000, lr=0.001000
Iter=4741120/epoch=12, Loss=0.000100, Triplet loss=1.416466, Training Accuracy=1.000000, lr=0.001000
Iter=4746240/epoch=12, Loss=0.000011, Triplet loss=1.372310, Training Accuracy=1.000000, lr=0.001000
Iter=4751360/epoch=12, Loss=0.000026, Triplet loss=1.305981, Training Accuracy=1.000000, lr=0.001000
Iter=4756480/epoch=12, Loss=0.000008, Triplet loss=1.416466, Training Accuracy=1.000000, lr=0.001000
Iter=4761600/epoch=12, Loss=0.000004, Triplet loss=1.261786, Training Accuracy=1.000000, lr=0.001000
Iter=4766720/epoch=12, Loss=0.000003, Triplet loss=0.665165, Training Accuracy=1.000000, lr=0.001000
Iter=4771840/epoch=12, Loss=0.000053, Triplet loss=1.240494, Training Accuracy=1.000000, lr

Iter=5120000/epoch=13, Validation Accuracy=0.991500, Precision=0.988666, Recall=0.994400
Iter=5125120/epoch=13, Loss=0.000001, Triplet loss=1.129204, Training Accuracy=1.000000, lr=0.001000
Iter=5130240/epoch=13, Loss=0.000005, Triplet loss=1.283884, Training Accuracy=1.000000, lr=0.001000
Iter=5135360/epoch=13, Loss=0.004909, Triplet loss=1.283884, Training Accuracy=1.000000, lr=0.001000
Iter=5140480/epoch=13, Loss=0.000047, Triplet loss=1.129219, Training Accuracy=1.000000, lr=0.001000
Iter=5145600/epoch=13, Loss=0.000007, Triplet loss=1.195495, Training Accuracy=1.000000, lr=0.001000
Iter=5150720/epoch=13, Loss=0.000004, Triplet loss=1.217592, Training Accuracy=1.000000, lr=0.001000
Iter=5155840/epoch=13, Loss=0.000001, Triplet loss=1.107107, Training Accuracy=1.000000, lr=0.001000
Iter=5160960/epoch=13, Loss=0.000010, Triplet loss=1.163511, Training Accuracy=1.000000, lr=0.001000
Iter=5166080/epoch=13, Loss=0.000004, Triplet loss=1.283884, Training Accuracy=1.000000, lr=0.001000
It

Iter=5524480/epoch=14, Loss=0.000340, Triplet loss=1.217592, Training Accuracy=1.000000, lr=0.001000
Iter=5529600/epoch=14, Loss=0.010962, Triplet loss=1.129204, Training Accuracy=1.000000, lr=0.001000
Iter=5529600/epoch=14, Validation Accuracy=0.977300, Precision=0.956746, Recall=0.999800
Iter=5534720/epoch=14, Loss=0.000019, Triplet loss=1.173398, Training Accuracy=1.000000, lr=0.001000
Iter=5539840/epoch=14, Loss=0.000195, Triplet loss=0.952427, Training Accuracy=1.000000, lr=0.001000
Iter=5544960/epoch=14, Loss=0.017908, Triplet loss=1.107107, Training Accuracy=0.984375, lr=0.001000
Iter=5550080/epoch=14, Loss=0.000339, Triplet loss=1.040825, Training Accuracy=1.000000, lr=0.001000
Iter=5555200/epoch=14, Loss=0.001007, Triplet loss=0.841942, Training Accuracy=1.000000, lr=0.001000
Iter=5560320/epoch=14, Loss=0.000001, Triplet loss=0.930330, Training Accuracy=1.000000, lr=0.001000
Iter=5565440/epoch=14, Loss=0.000042, Triplet loss=0.845579, Training Accuracy=1.000000, lr=0.001000
It